In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import os 
import json


kaggle_json = './drive/MyDrive/kaggle.json'
with open(kaggle_json, 'r') as f:
    json_data = json.load(f)
    os.environ['KAGGLE_USERNAME'] = json_data['username']
    os.environ['KAGGLE_KEY'] = json_data['key']


In [35]:
# 데이터 다운로드(api key 세팅 참조 링크 : https://github.com/Kaggle/kaggle-api)
!kaggle competitions download -c aerial-cactus-identification

aerial-cactus-identification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [36]:
import os

# 폴더 생성 (기존재시 pass)
os.makedirs('./input/train',  exist_ok=True)
os.makedirs('./input/test',  exist_ok=True)
os.makedirs('./output', exist_ok=True)

In [37]:
# 압축 해제 후 input 에 train.csv test.csv  넣기
import zipfile
zipfile.ZipFile(f'aerial-cactus-identification.zip').extractall('./input')

In [38]:
# 추가로 압축해제
zipfile.ZipFile(f'./input/test.zip').extractall('./input')
zipfile.ZipFile(f'./input/train.zip').extractall('./input')

In [39]:
import torch # 파이토치
import random
import numpy as np
import os

# 시드값 고정
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed) # 파이썬 난수 생성기 시드 고정
np.random.seed(seed) # 넘파이 난수 생성기 시드 고정
torch.manual_seed(seed) # 파이토치 난수 생성기 시드 고정(CPU 사용시)
torch.cuda.manual_seed(seed) # 파이토치 난수 생성기 시드 고정(GPU 사용시)
torch.cuda.manual_seed_all(seed) # 파이토치 난수 생성기 시드 고정(멀티 GPU 사용시)
torch.backends.cudnn.deterministic = True # 확정적 연산 사용
torch.backends.cudnn.benchmark = False # 벤치마크 기능 해제
torch.backends.cudnn.enabled = False # cudnn 사용해제

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [41]:
import pandas as pd

# 데이터 경로
labels = pd.read_csv('./input/train.csv')
submission = pd.read_csv('./input/sample_submission.csv')

In [10]:
from sklearn.model_selection import train_test_split

# 훈련데이터, 검증 데이터 분리
train, valid = train_test_split(labels,
                                test_size=0.1,
                                stratify=labels['has_cactus'],
                                random_state=50)

In [42]:
print('훈련 데이터 개수 :', len(train))
print('검증 데이터 개수 :', len(valid))

훈련 데이터 개수 : 15750
검증 데이터 개수 : 1750


In [43]:
import cv2 # openCV 라이브러리
from torch.utils.data import Dataset # 데이터 생성을 위한 클래스

class ImageDataset(Dataset):
  # 초기화 메서드(생성자)
  def __init__(self, df, img_dir='./', transform=None):
    super().__init__() # 상속받은 Dataset 생성자 호출
    # 전달받은 인수들 저장 
    self.df = df
    self.img_dir = img_dir
    self.transform = transform

  # 데이터셋 크기 반환 메서드
  def __len__(self):
    return len(self.df)

  # 인덱스(idx)에 해당하는 데이터 반환 메서드
  def __getitem__(self, idx):
    img_id = self.df.iloc[idx, 0] # 이미지 ID
    img_path = self.img_dir + img_id # 이미지 파일 경로
    image = cv2.imread(img_path) # 이미지 파일 읽기
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # 이미지 색상 보정
    label = self.df.iloc[idx, 1] # 이미지 레이블 (타깃값)

    if self.transform is not None:
      image = self.transform(image) # 변환기가 있다면 이미지 변환

    return image, label 


In [44]:
from torchvision import transforms # 이미지 변환을 위한 모듈

# 훈련 데이터용 변환기
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Pad(32, padding_mode='symmetric'),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(10),
    transforms.Normalize(
        (0.485, 0.456, 0.406),
        (0.229, 0.224, 0.225)
    ) 
     ])

# 검증 및 테스트 데이터용 변환기
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Pad(32, padding_mode='symmetric'),
    transforms.Normalize(
        (0.485, 0.456, 0.406),
        (0.229, 0.224, 0.225)
    )
])

In [45]:
dataset_train = ImageDataset(df = train, img_dir = './input/train/', transform = transform_train)
dataset_valid = ImageDataset(df = valid, img_dir = './input/train/', transform = transform_test)

In [46]:
from torch.utils.data import DataLoader

loader_train = DataLoader(dataset=dataset_train, batch_size=32, shuffle=True)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=32, shuffle=False)

**모델 생성**

In [47]:
import torch.nn as nn # 신경망 모듈
import torch.nn.functional as F # 신경망 모듈에서 자주 사용되는 함수

In [48]:
class Model(nn.Module):
  # 신경망 계층 정의
  def __init__(self):
    super().__init__() # 상속받은 nn.Module 의 __init__() 메서도 호출

    # 첫 번째 합성곱 계층
    self.layer1 = nn.Sequential(
      nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2),
      nn.BatchNorm2d(32), #배치 정규화
      nn.LeakyReLU(), # LeakyReLu 활성화 함수
      nn.MaxPool2d(kernel_size=2)  
    ) 
  
    # 두 번째 합성곱 계층
    self.layer2 = nn.Sequential(
      nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2),
      nn.BatchNorm2d(64),
      nn.LeakyReLU(),
      nn.MaxPool2d(kernel_size=2)
    )

    # 세 번째 합성곱 계층
    self.layer3 = nn.Sequential(
      nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2),
      nn.BatchNorm2d(128),
      nn.LeakyReLU(),
      nn.MaxPool2d(kernel_size=2)
    )

    # 네 번째 합성곱 계층
    self.layer4 = nn.Sequential(
      nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2),
      nn.BatchNorm2d(256),
      nn.LeakyReLU(),
      nn.MaxPool2d(kernel_size=2)
    )

    # 다섯 번째 합성곱 계층
    self.layer5 = nn.Sequential(
      nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2),
      nn.BatchNorm2d(512),
      nn.LeakyReLU(),
      nn.MaxPool2d(kernel_size=2)
    )
    
    self.avg_pool = nn.AvgPool2d(kernel_size=4)
    # 전결합 계층
    self.fc1 = nn.Linear(in_features=512 * 1 * 1, out_features=64)
    self.fc2 = nn.Linear(in_features=64, out_features=2)

  # 순전파 출력 정의
  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.layer5(x)
    x = self.avg_pool(x)
    x = x.view(-1, 512 * 1 * 1) # 평탄화
    x = self.fc1(x)
    x = self.fc2(x)
    return x

In [49]:
model = Model().to(device)
model

Model(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Sequential(
    (0): Conv2d(128,

In [50]:
# 손실 함수 설정
criterion = nn.CrossEntropyLoss()

In [52]:
# 옵티마이저 설정
optimizer = torch.optim.Adamax(model.parameters(), lr=0.00006)

In [53]:
epochs = 70 # 총 에폭
# 총 에폭만큼 반폭 

for epoch in range(epochs):
  epoch_loss = 0 # 에폭별 손실값 최소화

  # '반복 횟수' 만큼 반복
  for images, labels in loader_train:
    # 이미지, 레이블 데이터 미니배치를 장비에 할당
    images = images.to(device)
    labels = labels.to(device)

    # 옵티마이저 내 기울기 초기화
    optimizer.zero_grad()

    # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
    outputs = model(images)

    # 손실 함수를 활용해 outpus 와 labels 의 손실값 계산
    loss = criterion(outputs, labels)

    # 현재 배치에서의 손실값 추가
    epoch_loss += loss.item() # 역전파 수행
    loss.backward()
    # 가중치 갱신
    optimizer.step()

  # 훈련 데이터 손실값 출력
  print(f'에폭 [{epoch+1}/{epochs}] - 손실값: {epoch_loss / len(loader_train):.4f}')

에폭 [1/70] - 손실값: 0.1287
에폭 [2/70] - 손실값: 0.0657
에폭 [3/70] - 손실값: 0.0480
에폭 [4/70] - 손실값: 0.0416
에폭 [5/70] - 손실값: 0.0337
에폭 [6/70] - 손실값: 0.0346
에폭 [7/70] - 손실값: 0.0314
에폭 [8/70] - 손실값: 0.0270
에폭 [9/70] - 손실값: 0.0251
에폭 [10/70] - 손실값: 0.0260
에폭 [11/70] - 손실값: 0.0228
에폭 [12/70] - 손실값: 0.0228
에폭 [13/70] - 손실값: 0.0195
에폭 [14/70] - 손실값: 0.0206
에폭 [15/70] - 손실값: 0.0200
에폭 [16/70] - 손실값: 0.0172
에폭 [17/70] - 손실값: 0.0158
에폭 [18/70] - 손실값: 0.0171
에폭 [19/70] - 손실값: 0.0164
에폭 [20/70] - 손실값: 0.0162
에폭 [21/70] - 손실값: 0.0137
에폭 [22/70] - 손실값: 0.0147
에폭 [23/70] - 손실값: 0.0124
에폭 [24/70] - 손실값: 0.0116
에폭 [25/70] - 손실값: 0.0135
에폭 [26/70] - 손실값: 0.0113
에폭 [27/70] - 손실값: 0.0123
에폭 [28/70] - 손실값: 0.0127
에폭 [29/70] - 손실값: 0.0114
에폭 [30/70] - 손실값: 0.0111
에폭 [31/70] - 손실값: 0.0096
에폭 [32/70] - 손실값: 0.0097
에폭 [33/70] - 손실값: 0.0100
에폭 [34/70] - 손실값: 0.0100
에폭 [35/70] - 손실값: 0.0091
에폭 [36/70] - 손실값: 0.0097
에폭 [37/70] - 손실값: 0.0073
에폭 [38/70] - 손실값: 0.0074
에폭 [39/70] - 손실값: 0.0069
에폭 [40/70] - 손실값: 0.0101
에폭 [41/70

In [54]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수 임포트 

# 실젯값 과 예측 확률값을 담을 리스트 
true_list = []
preds_list = []

model.eval()

with torch.no_grad():
  for images, labels in loader_valid:
    # 이미지, 레이블 데이터 미니배치를 장비에 할당
    images = images.to(device)
    labels = labels.to(device)

    # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 줄력값 계산
    outputs = model(images)
    preds = torch.softmax(outputs.cpu(), dim=1)[:,1]
    true = labels.cpu()

    # 예측확률과 실젯값을 리스트에 추가
    preds_list.extend(preds)
    true_list.extend(true)

print(f'검증 데이터 ROC AUC : {roc_auc_score(true_list, preds_list):.4f}')


검증 데이터 ROC AUC : 0.9998


In [55]:
dataset_test = ImageDataset(df=submission, img_dir ='./input/test/', transform=transform_test)
loader_test = DataLoader(dataset=dataset_test, batch_size=32, shuffle=False)

In [56]:
model.eval()

preds = [] # 타깃 예측값 저장용 리스트 초기화

with torch.no_grad():
  for images, _ in loader_test:
    # 이미지 데이터 미니배치를 장비에 할당
    images = images.to(device)

    # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 줄력값 계산
    outputs = model(images)
    # 타깃값 1일 확률 
    preds_part = torch.softmax(outputs.cpu(), dim=1)[:,1].tolist()

    # 예측확률 리스트에 추가
    preds.extend(preds_part)


In [57]:
submission['has_cactus'] = preds
submission.to_csv('./output/submission.csv', index=False)

In [58]:
!kaggle competitions submit -c aerial-cactus-identification -f ./output/submission.csv -m submit_in_colab

100% 188k/188k [00:01<00:00, 124kB/s]
400 - Bad Request
